# Collaboration with Yuri @Ziv's lab
Dyskinesia progerssion GWAS


#### predictors
GWAS and some targeted SNPs + GBA, LRRK2 score analyses



#### Strata
All sample analysis, male only analysis, female only analysis


#### Models
no adjustment - only PC1-3. adjustment - plus those in [ ] 

Linear: Disease_duration(the duration to the first Dyskinesia==1)~SNP+PC1+PC2+PC3 [+PD_AAO+Sex+HY(ExceptForCORIELL):for ADJ]

Lotistic: Dyskinesia~SNP+PC1+PC2+PC3 [+Disease_duration+PD_AAO+Sex+HY(ExceptForCORIELL):for ADJ]

Survival: Dyskinesia~SNP+PC1+PC2+PC3 [+PD_AAO+Sex+HY(ExceptForCORIELL):for ADJ] + Time

In [1]:
# Preparation
import glob
import os
import subprocess
import pandas as pd
import subprocess
import shutil
import time
import numpy as np
import sys
from IPython.core.debugger import set_trace # insert 'set_truce()' inside function for debagging
import matplotlib as plt

# Some job control commands
def submitMultiJobs(x, time='3:00:00', mem=2, bundle=1, modules=None, store='swarm', node='quick', packing=True, go=False):
    """
    Submit multiple jobs on biowulf
    x: LIST of the location of swarmfile
    max runtime: quick->4:00:00, norm->48:00:00 be careful when bundle jobs
    node: quick or norm
    example of modules: modeule='R,plink'
    'go=True' will submit the jobs
    """
    swarmdevq="swarm -f {F} --time={T} -g {G} -p {PACK} -b {B} --logdir {L}{M} --partition={P} --devel"
    a=[]
    MODULES=' --module {M}'.format(M=modules)*(modules is not None)
    for i in x: 
        s=swarmdevq.format(F=i, T=time, G=mem, PACK=1+packing, B=bundle, M=MODULES, L=store, P=node)
        if go:
            res=subprocess.run(s[:-8].split(" "), stdout=subprocess.PIPE)
            a.append(res.stdout.decode('utf-8'))
        if not go:
            res=subprocess.run(s.split(" "), stdout=subprocess.PIPE)
            t = res.stdout.decode('utf-8').replace(',', '--').replace('\n', '--').replace(' /spi', '--').split("--")
            x = [i for i,x in enumerate(t) if 'cpus-' in x]
            print(t[1:3]+t[x[0]:-2])
            a='Check!![module, N_jobs, node, mem(x2), and store,bundle] If ok -> go=True!'
    return(a)


def checkJobStatus(list_of_jobids):
    """
    Get the job status as the list of dataframe with some outputs
    """
    ans = []
    index=0
    for i in list_of_jobids:
        res = subprocess.run(['jobhist', str(i)], stdout=subprocess.PIPE)
        res2 = res.stdout.decode().split('\n')
        start = next(i for i,x in enumerate(res2) if 'Jobid' in x)
        start_script = next(i for i,x in enumerate(res2) if '/usr/local/bin/swarm -f' in x)
        script = res2[start_script].replace('Swarm Command      : /usr/local/bin/swarm -f ', '').split('--')[0]
        s = pd.Series(res2[start:])
        df = s.str.split("\\s{2,}", expand=True)
        df = df.rename(columns=df.iloc[0, :])
        a=df[1:-2]
        print('=========== index=' + str(index) + ": jobID=" + i + ": " + script + ' ===============')
        print("N of Jobs: " + str(a.shape[0]))
        print('N_of not_COMPLETED: '+str(sum(a.State!='COMPLETED')))
        print("First 5 of longest Runtime job")
        print(a.sort_values('Runtime', ascending=False).head())
        print("Pending jobs"+str(list(a[a.State=='PENDING'].Jobid)))
        ans.append(a)
        index += 1
    return (ans)

def plotRuntime(Runtime):
    """
    Create plot from Outputted Runtime from checkJobStatus
    """
    def convertH(runtime):
        h, m, s = runtime.split(':')
        return int(h) + int(m) / 60 + int(s)/3600
    Runtime.apply(convertH).plot.hist()

def checkSubjob(subjobID):
    """
    Get the command of subjobs to check. 
    Provide subjobID in string
    """
    header='/spin1/swarm/iwakih2/'
    findfolder=header+subjobID.replace("_", '/cmd.')
    files = glob.glob(findfolder+'_*')
    for file in files:
        res = subprocess.run(['cat', file], stdout=subprocess.PIPE)
        return(print(res.stdout.decode()))

def retrieveTimeouts(x, output):
    """
    Writing timeouted jobs in 'output'
    x is the output dataframe from "checkJobStatus"
    """
    failed=x[x.State=='TIMEOUT'].Jobid
    header='/spin1/swarm/iwakih2/'
    findlist=[header+i.replace("_", '/cmd.') for i in failed]
    with open(output, 'w') as f:
        for i in findlist:
            files = glob.glob(i+'_*')
            for file in files:
                res = subprocess.run(['cat', file], stdout=subprocess.PIPE)
                f.write(res.stdout.decode()[3:-2] + '\n')
def cutSwarmByNjobs(swarmfile, N=2000):    
    a = []
    with open(swarmfile) as f:
        lines = f.read().splitlines()
        N_split = len(lines)//N
        for i in range(N_split+1):
            fname,ext = swarmfile.split('.')
            newfile='{0}_{1}cut{2}.{3}'.format(fname, str(N), str(i), ext)
            a.append(newfile)
            with open(newfile, 'w') as f:
                f.write('\n'.join(lines[(i*N):((i+1)*N)]))
    return (a)

def submitTerminal(command, printing=False, message=''):
    # quick command to submit jobs to terminal
    start = time.time()
    res=subprocess.run(command.split(' '), stdout=subprocess.PIPE)
    end = time.time()
    sys.stdout.write('EXEC_TIME in sec: '+ str(round(end - start, 3)) + ' : ')
    if printing:
        print(res.stdout.decode('utf-8'))
    if message=='':
        return(res.stdout.decode('utf-8'))
    else:
        print(message, '\n')

In [2]:
DATASET=["CORIELL", "PRECEPT", "SCOPA"]
OUTPUT="cohort/"
CHRNUM=['chr'+str(i) for i in range(1,23)]
IMPUTED='/data/LNG/CORNELIS_TEMP/progression_GWAS/'

In [3]:
# Cuts per 20K based on maf 0.01 and rsq 0.3 were previously conducted. chack the number of cutted files.
for i in DATASET:
    check= "ls /data/LNG/iwakih2/dataset/{dataset}/maf01rsq3_20Kcut/"
    bash =check.format(dataset=i)
    res=subprocess.run(bash.split(" "), stdout=subprocess.PIPE)
    num=res.stdout.decode('utf-8').count('gz')
    print(i, num)

CORIELL 153
PRECEPT 149
SCOPA 399


In [5]:
# two different arrays
NEUROX=['CORIELL', 'PRECEPT']
CHIP=list(set(DATASET)-set(NEUROX))

In [6]:
# check wether we missed some files of not
df=pd.DataFrame()
for i in NEUROX:
    check= "ls /data/LNG/iwakih2/dataset/{dataset}/maf01rsq3_20Kcut/"
    bash =check.format(dataset=i)
    res=subprocess.run(bash.split(" "), stdout=subprocess.PIPE)
    files = [i for i in res.stdout.decode('utf-8').split('\n') if 'gz' in i]
    df = df.append(pd.DataFrame(data = {'dataset':i, 'file':files, 'yes':"O"}))
tab=df.pivot(index='file', columns='dataset', values='yes')
# By manual checks, no missing data (only different based on number of variants selected by maf and rsq thres)
tab[tab.index.str.contains('cut10.', regex=False)]

dataset,CORIELL,PRECEPT
file,,
cut10.0.txt.gz,O,O
cut10.1.txt.gz,O,O
cut10.2.txt.gz,O,O
cut10.3.txt.gz,O,O
cut10.4.txt.gz,O,O
cut10.5.txt.gz,O,O
cut10.6.txt.gz,O,O


In [7]:
# Check if the maf01rsq3.info is ok
for i in NEUROX:
    check= "wc -l /data/LNG/iwakih2/dataset/{dataset}/maf01rsq3.info"
    bash =check.format(dataset=i)
    res=subprocess.run(bash.split(" "), stdout=subprocess.PIPE)
    num=res.stdout.decode('utf-8')
    print(i, num)

CORIELL 2815231 /data/LNG/iwakih2/dataset/CORIELL/maf01rsq3.info

PRECEPT 2781979 /data/LNG/iwakih2/dataset/PRECEPT/maf01rsq3.info



In [8]:
for i in CHIP:
    check= "wc -l /data/LNG/iwakih2/dataset/{dataset}/maf01rsq3.info"
    bash =check.format(dataset=i)
    res=subprocess.run(bash.split(" "), stdout=subprocess.PIPE)
    num=res.stdout.decode('utf-8')
    print(i, num)

SCOPA 7767315 /data/LNG/iwakih2/dataset/SCOPA/maf01rsq3.info



In [9]:
# Derive genetic ID to match the clinical ID and genetic ID
df = pd.DataFrame()
for i in DATASET:
    fam_place="/data/LNG/CORNELIS_TEMP/progression_GWAS/{dataset}/plink_files_hard/"
    fam=glob.glob(fam_place.format(dataset=i)+"*.fam")
    t = pd.read_table(fam[0], delim_whitespace=True, header=None)
    t['Cohort']=i
    print(i, t.shape)
    df = df.append(t)
print('All', df.shape)
!mkdir -p data
df.to_csv("data/nonPPMI_geneticIDs.txt", index=False, header=False)

CORIELL (340, 7)
PRECEPT (318, 7)
SCOPA (291, 7)
All (949, 7)


# Phenotype

Created by R

# Analysis

When importing vcf the default setting doesn't import GENOTYPED variants as their INFO is PASS;GENOTYPED. The argument to get genotyped variants is **--var-filter PASS . GENOTYPED**


In [107]:
# logistic
a = []

DATASET = 'SCOPA'

    # Create pfile for each chromosome

for i in range(1,23):

    CHRNUM = f'chr{i}'
    script1 = [f"""\
plink2 \
 --vcf '/data/CARD/PD/imputed_data/{DATASET}/{CHRNUM}.dose.vcf.gz' 'dosage=DS' \
 --var-filter PASS . GENOTYPED\
 --extract-if-info R2'>'=0.3 \
 --mac 2 \
 --make-pgen --out /data/CARD/projects/dysk_prog/logistic/{DATASET}.{CHRNUM} \
"""]
    
    # Analysis

    for SEX in ['MALE', 'FEMALE', 'ALL']:
        for MODEL in ['ADJ', 'NOADJ']:
            
            # define COV

            if MODEL=='NOADJ':
                COVS = 'PC1,PC2,PC3'
            elif SEX=='ALL':
                COVS = 'PD_AAO,Sex,HY,Disease_duration,PC1,PC2,PC3'
            else:
                COVS = 'PD_AAO,HY,Disease_duration,PC1,PC2,PC3'
                
            script1.append(f"""\
plink2 --pfile /data/CARD/projects/dysk_prog/logistic/{DATASET}.{CHRNUM} \
 --glm 'no-firth' 'hide-covar' \
 --freq \
 --covar-variance-standardize \
 --ci 0.95 \
 --pheno data/{DATASET}_{SEX}.txt 'iid-only' --pheno-name Dyskinesia \
 --covar data/{DATASET}_{SEX}.txt 'iid-only' --covar-name {COVS} \
 --out /data/CARD/projects/dysk_prog/logistic/{DATASET}.{SEX}.{MODEL}.{CHRNUM} \
""")
        
    a.append(' ; '.join(script1))

with open('swarm_logistics.txt', 'w') as f:
    f.write('\n'.join(a))

In [106]:
!wc -l /data/CARD/projects/dysk_prog/logistic/SCOPA.ALL.ADJ.chr1.Dyskinesia.glm.logistic

688559 /data/CARD/projects/dysk_prog/logistic/SCOPA.ALL.ADJ.chr1.Dyskinesia.glm.logistic


In [108]:
submitMultiJobs(['swarm_logistics.txt'], modules='plink/2.3-alpha', time='0:10:00', go=True)

['30165593\n']

In [120]:
t = checkJobStatus(['30165593'])

=========== index=0: jobID=30165593: swarm_logistics.txt  ===============
N of Jobs: 11
N_of not_COMPLETED: 0
First 5 of longest Runtime job
        Jobid Partition      State Nodes CPUs  Walltime   Runtime      MemReq  \
1  30165593_0     quick  COMPLETED     1    2  00:10:00  00:05:03  4.0GB/node   
2  30165593_1     quick  COMPLETED     1    2  00:10:00  00:04:26  4.0GB/node   
3  30165593_2     quick  COMPLETED     1    2  00:10:00  00:03:59  4.0GB/node   
6  30165593_5     quick  COMPLETED     1    2  00:10:00  00:03:38  4.0GB/node   
4  30165593_3     quick  COMPLETED     1    2  00:10:00  00:03:34  4.0GB/node   

  MemUsed Nodelist  
1   2.2GB   cn2864  
2   2.4GB   cn2864  
3   2.4GB   cn2864  
6   2.1GB   cn2656  
4   2.1GB   cn2864  
Pending jobs[]


In [114]:
# linear
a = []

for DATASET in ['SCOPA', 'CORIELL', 'PRECEPT']:
    
    # Create pfile for each chromosome

    for i in range(1,23):

        CHRNUM = f'chr{i}'
        script1 = [f"""\
plink2 \
 --vcf '/data/CARD/PD/imputed_data/{DATASET}/{CHRNUM}.dose.vcf.gz' 'dosage=DS' \
 --var-filter PASS . GENOTYPED\
 --extract-if-info R2'>'=0.3 \
 --mac 2 \
 --make-pgen --out /data/CARD/projects/dysk_prog/linear/{DATASET}.{CHRNUM} \
"""]

        # Analysis

        for SEX in ['MALE', 'FEMALE', 'ALL']:
            for MODEL in ['ADJ', 'NOADJ']:

                # define COV

                if MODEL=='NOADJ':
                    COVS = 'PC1,PC2,PC3'
                elif SEX=='ALL':
                    COVS = 'PD_AAO,Sex,HY,PC1,PC2,PC3'
                else:
                    COVS = 'PD_AAO,HY,PC1,PC2,PC3'
                
                # CORIELL doesn't have HY
                if DATASET=='CORIELL':
                    COVS = COVS.replace('HY,','')

                script1.append(f"""\
plink2 --pfile /data/CARD/projects/dysk_prog/linear/{DATASET}.{CHRNUM} \
 --glm 'no-firth' 'hide-covar' \
 --freq \
 --covar-variance-standardize \
 --pheno data/{DATASET}_{SEX}.lin.txt 'iid-only' --pheno-name Disease_duration \
 --covar data/{DATASET}_{SEX}.lin.txt 'iid-only' --covar-name {COVS} \
 --out /data/CARD/projects/dysk_prog/linear/{DATASET}.{SEX}.{MODEL}.{CHRNUM} \
""")

        a.append(' ; '.join(script1))

with open('swarm_lin.txt', 'w') as f:
    f.write('\n'.join(a))

In [115]:
submitMultiJobs(['swarm_lin.txt'], modules='plink/2.3-alpha', time='0:10:00', mem='8', go=True)

['30167102\n']

In [129]:
t = checkJobStatus(['30167102'])

=========== index=0: jobID=30167102: swarm_lin.txt  ===============
N of Jobs: 33
N_of not_COMPLETED: 0
First 5 of longest Runtime job
          Jobid Partition      State Nodes CPUs  Walltime   Runtime  \
12  30167102_11     quick  COMPLETED     1    2  00:10:00  00:04:21   
23  30167102_22     quick  COMPLETED     1    2  00:10:00  00:04:13   
13  30167102_12     quick  COMPLETED     1    2  00:10:00  00:03:51   
24  30167102_23     quick  COMPLETED     1    2  00:10:00  00:03:50   
14  30167102_13     quick  COMPLETED     1    2  00:10:00  00:03:30   

         MemReq MemUsed Nodelist  
12  16.0GB/node   2.6GB   cn2656  
23  16.0GB/node   2.4GB   cn2847  
13  16.0GB/node   2.6GB   cn2656  
24  16.0GB/node   2.3GB   cn2847  
14  16.0GB/node   2.5GB   cn2656  
Pending jobs[]


In [70]:
# Survival

with open('swarm_surv.txt', 'w') as f:
    for DATASET in ['SCOPA', 'CORIELL', 'PRECEPT']:
        infofolder=f"/data/LNG/iwakih2/dataset/{DATASET}/maf01rsq3_20Kcut"
        FILES = glob.glob(f'{infofolder}/cut*txt.gz',)

        for FILE in FILES:
            for SEX in ['ALL']:
                for MODEL in ['ADJ', 'NOADJ']:

                    # define COV

                    if MODEL=='NOADJ':
                        COVS = 'PC1+PC2+PC3'
                    else:
                        COVS = 'PD_AAO+Sex+HY+PC1+PC2+PC3'

                    # CORIELL doesn't have HY
                    if DATASET=='CORIELL':
                        COVS = COVS.replace('HY+','')


                    line = f"Rscript --vanilla surv.R '{MODEL};Dyskinesia;{COVS};{FILE};data/{DATASET}_{SEX}.surv.txt;/data//CARD/projects/dysk_prog/surv/{DATASET}'\n"
                    f.write(line) 

In [78]:
submitMultiJobs(['swarm_surv.txt'], modules='R/4.1.0', time='3:00:00', mem=4, go=True)

['27598592\n']

In [87]:
t = checkJobStatus(['27598592'])

=========== index=0: jobID=27598592: swarm_surv.txt  ===============
N of Jobs: 701
N_of not_COMPLETED: 9
First 5 of longest Runtime job
            Jobid Partition      State Nodes CPUs  Walltime   Runtime  \
74    27598592_73     quick  COMPLETED     1    2  03:00:00  00:19:07   
119  27598592_118     quick  COMPLETED     1    2  03:00:00  00:18:55   
115  27598592_114     quick  COMPLETED     1    2  03:00:00  00:18:48   
179  27598592_178     quick  COMPLETED     1    2  03:00:00  00:18:47   
79    27598592_78     quick  COMPLETED     1    2  03:00:00  00:18:44   

         MemReq MemUsed Nodelist  
74   8.0GB/node   0.6GB   cn2616  
119  8.0GB/node   0.6GB   cn2870  
115  8.0GB/node   0.6GB   cn2870  
179  8.0GB/node   0.6GB   cn2606  
79   8.0GB/node   0.6GB   cn2616  
Pending jobs[]


In [121]:
# Survival (Male, Female version) ## Added 

with open('swarm_surv2.txt', 'w') as f:
    for DATASET in ['SCOPA', 'CORIELL', 'PRECEPT']:
        infofolder=f"/data/LNG/iwakih2/dataset/{DATASET}/maf01rsq3_20Kcut"
        FILES = glob.glob(f'{infofolder}/cut*txt.gz',)

        for FILE in FILES:
            for SEX in ['MALE', 'FEMALE']:
                for MODEL in ['ADJ', 'NOADJ']:

                    # define COV

                    if MODEL=='NOADJ':
                        COVS = 'PC1+PC2+PC3'
                    else:
                        COVS = 'PD_AAO+HY+PC1+PC2+PC3'

                    # CORIELL doesn't have HY
                    if DATASET=='CORIELL':
                        COVS = COVS.replace('HY+','')


                    line = f"Rscript --vanilla surv.R '{MODEL};Dyskinesia;{COVS};{FILE};data/{DATASET}_{SEX}.surv.txt;/data//CARD/projects/dysk_prog/surv/{DATASET}'\n"
                    f.write(line) 

In [134]:
submitMultiJobs(['swarm_surv2.txt'], modules='R/4.1.0', time='1:00:00', mem=4,bundle=2,  go=True)

['30169626\n']

In [137]:
t = checkJobStatus(['30169626'])

=========== index=0: jobID=30169626: swarm_surv2.txt  ===============
N of Jobs: 701
N_of not_COMPLETED: 0
First 5 of longest Runtime job
            Jobid Partition      State Nodes CPUs  Walltime   Runtime  \
567  30169626_566     quick  COMPLETED     1    2  02:00:00  00:37:17   
548  30169626_547     quick  COMPLETED     1    2  02:00:00  00:37:16   
547  30169626_546     quick  COMPLETED     1    2  02:00:00  00:37:13   
550  30169626_549     quick  COMPLETED     1    2  02:00:00  00:36:58   
552  30169626_551     quick  COMPLETED     1    2  02:00:00  00:36:57   

         MemReq MemUsed Nodelist  
567  8.0GB/node   0.6GB   cn0724  
548  8.0GB/node   0.6GB   cn0704  
547  8.0GB/node   0.6GB   cn0704  
550  8.0GB/node   0.6GB   cn0704  
552  8.0GB/node   0.6GB   cn0704  
Pending jobs[]


# Tareget analyses

For 90 risk snps and GBA, LRRK2 scores

90 risk snps were already derived in the previous research. Need to create GBA, LRRK2 scores

*Analyses on specific genes focus on GBA and LRRK2 variants associated with PD increased/decreased risk. The carrier status of these variants are collapsed into three single variables: carrier status of GBA risk variants, carrier status of LRRK2 risk variants and carrier status of LRRK2 protective haplotype. The variants that have been included are: T369M, E326K, N370S and L444P for GBA; G2019S, M1646T and R1441C/G for LRRK2 risk variants and R1398H as a tagging variant for LRRK2 protective haplotype.*

In [141]:
# Target 2 adding the cumulative GBA, LRRK2 scores to PRS90 and META5 PRS
GBAs = {0:['rs2230288', 'E326K', '1:155206167', 'T'],
        1:['rs76763715', 'N370S', '1:155205634', 'C'],
        2:['rs75548401', 'T369zM','1:155206037', 'A'],
        3:['rs1569861490', 'L444P', '1:12064609', 'C']}
LRRK2s = {0:['rs34637584', 'G2010S', '12:40734202', 'A'],
          1:['rs35303786', 'M1646T', '12:40713899', 'C'],
          2:['rs33939927', 'R1441C', '12:13715851', 'T'],
          3:['rs33939927', 'R1441G', '12:13715851', 'G'],
          4:['rs7133914', 'R1398H', '12:40702911', 'A']} # protective

GBAs = pd.DataFrame.from_dict(GBAs, orient='index', columns=['rsid','name', 'ID', 'alt'])
LRRK2s=pd.DataFrame.from_dict(LRRK2s, orient='index', columns=['rsid','name', 'ID', 'alt'])
GBAs[['ID', 'alt']].to_csv('chr1.target.txt', sep='\t', index=False, header=False)
LRRK2s[['ID', 'alt']].to_csv('chr12.target.txt', sep='\t', index=False, header=False)

In [142]:
!mkdir /data/CARD/projects/dysk_prog/target2

mkdir: cannot create directory '/data/CARD/projects/dysk_prog/target2': File exists


In [177]:
# Target2
for DATASET in ['SCOPA', 'CORIELL', 'PRECEPT']:
    df_prs = pd.read_csv(f'/data/LNG/iwakih2/dataset/{DATASET}/target/all.txt', sep='\t', index_col='IID')
    a = [df_prs]
    # Create pfile for each chromosome
    
    for i in [1,12]:
        CHRNUM = f'chr{i}'
        
        # derive variants from vcf
        df = pd.read_csv(f'{CHRNUM}.target.txt', header=None, sep='\t', names=['ID', 'Alt'])
        script1 = f"""\
bcftools view -r {",".join(df.ID.unique())} /data/CARD/PD/imputed_data/{DATASET}/{CHRNUM}.dose.vcf.gz\
 -Oz -o /data/CARD/projects/dysk_prog/target2/{DATASET}.{CHRNUM}.vcf.gz ; \
"""
        t=submitTerminal(script1)
        
        
        # create raw file with INFO filter
        script2 = f"""\
plink2\
 --vcf /data/CARD/projects/dysk_prog/target2/{DATASET}.{CHRNUM}.vcf.gz dosage=DS\
 --var-filter PASS . GENOTYPED\
 --extract-if-info R2>=0.3\
 --export A --out /data/CARD/projects/dysk_prog/target2/{DATASET}.{CHRNUM}\
 --export-allele {CHRNUM}.target.txt"""
        t=submitTerminal(script2)
        
        
        if CHRNUM=='chr1':
            # GBA
            d = pd.read_csv(f'/data/CARD/projects/dysk_prog/target2/{DATASET}.{CHRNUM}.raw', sep='\t')
            relevant_columns = np.intersect1d(GBAs.ID+'_'+GBAs.alt, d.columns)
            if len(relevant_columns)>0:
                d['GBAr_N_A'] = (d[relevant_columns].sum(axis=1)>0)*1
                # Usually SNP_REF_ALT but since this is not SNP, give _N_A instead.
            else:
                d['GBAr_N_A'] = np.nan
            
            a.append(d[['IID', 'GBAr_N_A']].set_index('IID'))
            
            
        if CHRNUM=='chr12':
            # LRRK2 risk
            d = pd.read_csv(f'/data/CARD/projects/dysk_prog/target2/{DATASET}.{CHRNUM}.raw', sep='\t')
            relevant_columns = np.intersect1d(LRRK2s.ID+'_'+LRRK2s.alt, d.columns).tolist()
            relevant_columns.remove('12:40702911_A')
            if len(relevant_columns)>0:
                d['LRRK2r_N_A'] = (d[relevant_columns].sum(axis=1)>0)*1
            else:
                d['LRRK2r_N_A'] = np.nan

            # LRRK2 protective
            if d.columns.isin(['12:40702911_A']).sum().sum()>0:
                d['LRRK2p_N_A'] = (d['12:40702911_A']>0)*1
            else:
                d['LRRK2p_N_A'] = np.nan
            
            a.append(d[['IID', 'LRRK2r_N_A', 'LRRK2p_N_A']].set_index('IID'))
        
        
        
        pd.concat(a, axis=1).to_csv(f'/data/CARD/projects/dysk_prog/target2/{DATASET}.all.txt', sep='\t')


EXEC_TIME in sec: 0.044 : EXEC_TIME in sec: 0.013 : EXEC_TIME in sec: 0.037 : EXEC_TIME in sec: 0.011 : EXEC_TIME in sec: 0.03 : EXEC_TIME in sec: 0.011 : EXEC_TIME in sec: 0.038 : EXEC_TIME in sec: 0.011 : EXEC_TIME in sec: 0.029 : EXEC_TIME in sec: 0.011 : EXEC_TIME in sec: 0.036 : EXEC_TIME in sec: 0.011 : 

In [ ]:
# Survival Aanalysis - Target
for DATASET in ['SCOPA', 'CORIELL', 'PRECEPT']:
    infofolder=f"/data/LNG/iwakih2/dataset/{DATASET}/maf01rsq3_20Kcut"
    FILES = [f"/data/CARD/projects/dysk_prog/target2/{DATASET}.all.txt"]

    for FILE in FILES:
        for SEX in ['ALL', 'MALE', 'FEMALE']:
            for MODEL in ['ADJ', 'NOADJ']:

                # define COV

                if MODEL=='NOADJ':
                    COVS = 'PC1+PC2+PC3'
                    
                elif SEX=='ALL':
                    COVS = 'PD_AAO+Sex+HY+PC1+PC2+PC3'
                else:
                    COVS = 'PD_AAO+HY+PC1+PC2+PC3'
                
                # CORIELL doesn't have HY
                if DATASET=='CORIELL':
                    COVS = COVS.replace('HY+','')
                    
                # survival
                line = f"Rscript --vanilla surv_target.R {MODEL};Dyskinesia;{COVS};{FILE};data/{DATASET}_{SEX}.surv.txt;/data/CARD/projects/dysk_prog/target2/result"
                print(line)
                t=submitTerminal(line)
                
                # linear
                line = f"Rscript --vanilla lin_target.R {MODEL};Disease_duration;{COVS};{FILE};data/{DATASET}_{SEX}.lin.txt;/data/CARD/projects/dysk_prog/target2/result"
                print(line)
                t=submitTerminal(line)
                
                # logistic
                if DATASET=='SCOPA':
                    COVS=COVS + '+Disease_duration'
                    line = f"Rscript --vanilla logis_target.R {MODEL};Dyskinesia;{COVS};{FILE};data/{DATASET}_{SEX}.txt;/data/CARD/projects/dysk_prog/target2/result"
                    print(line)
                    t=submitTerminal(line)


In [122]:
# !rm /data/CARD/projects/dysk_prog/target2/lgs/*
!ls /data/CARD/projects/dysk_prog/target2/result/

CORIELL.ALL.ADJ.Disease_duration.lin.txt
CORIELL.ALL.ADJ.Dyskinesia.cox.txt
CORIELL.ALL.NOADJ.Disease_duration.lin.txt
CORIELL.ALL.NOADJ.Dyskinesia.cox.txt
CORIELL.FEMALE.ADJ.Disease_duration.lin.txt
CORIELL.FEMALE.ADJ.Dyskinesia.cox.txt
CORIELL.FEMALE.NOADJ.Disease_duration.lin.txt
CORIELL.FEMALE.NOADJ.Dyskinesia.cox.txt
CORIELL.MALE.ADJ.Disease_duration.lin.txt
CORIELL.MALE.ADJ.Dyskinesia.cox.txt
CORIELL.MALE.NOADJ.Disease_duration.lin.txt
CORIELL.MALE.NOADJ.Dyskinesia.cox.txt
PRECEPT.ALL.ADJ.Disease_duration.lin.txt
PRECEPT.ALL.ADJ.Dyskinesia.cox.txt
PRECEPT.ALL.NOADJ.Disease_duration.lin.txt
PRECEPT.ALL.NOADJ.Dyskinesia.cox.txt
PRECEPT.FEMALE.ADJ.Disease_duration.lin.txt
PRECEPT.FEMALE.ADJ.Dyskinesia.cox.txt
PRECEPT.FEMALE.NOADJ.Disease_duration.lin.txt
PRECEPT.FEMALE.NOADJ.Dyskinesia.cox.txt
PRECEPT.MALE.ADJ.Disease_duration.lin.txt
PRECEPT.MALE.ADJ.Dyskinesia.cox.txt
PRECEPT.MALE.NOADJ.Disease_duration.lin.txt
PRECEPT.MALE.NOADJ.Dyskinesia.cox.txt
SCOPA.ALL.ADJ.Disease_duration.l

# Combine results

Combine the GWAS results and save them into "RES" folder

In [131]:
!mkdir /data/CARD/projects/dysk_prog/RES

In [132]:
%%bash
for DATASET in  CORIELL SCOPA PRECEPT ;do
    for SEX in ALL MALE FEMALE ;do
        for MODEL in ADJ NOADJ ;do
            TAIL=Dyskinesia.cox.txt
            awk 'FNR==1 && NR!=1{next;}{print}' /data/CARD/projects/dysk_prog/surv/${DATASET}/${DATASET}.${SEX}.${MODEL}.*.${TAIL} > /data/CARD/projects/dysk_prog/RES/${DATASET}.${SEX}.${MODEL}.${TAIL}
        done
    done
done

In [133]:
def combine_lin_outputs(DATASET, SEX, MODEL):
    d = pd.DataFrame()
    for i in range(1,23):
        
        d1 = pd.read_csv(f'/data/CARD/projects/dysk_prog/linear/{DATASET}.{SEX}.{MODEL}.chr{i}.Disease_duration.glm.linear', sep='\t')
        d2 = pd.read_csv(f'/data/CARD/projects/dysk_prog/linear/{DATASET}.{SEX}.{MODEL}.chr{i}.afreq', sep='\t')
        if d1.shape[0] != d2.shape[0]:
            print(f'ERROR for {DATASET}.{SEX}.{MODEL}.chr{i}')
        else:
            d1['ALT_Frq'] = d2.ALT_FREQS
            d1['BETA'] = d1.BETA.where(d1.ALT==d1.A1, other=-d1.BETA)
            d1['T_STAT'] = d1.T_STAT.where(d1.ALT==d1.A1, other=-d1.T_STAT)
            d1 = d1.drop(columns=['A1'])
            d = d.append(d1)
    return(d)

In [134]:
for DATASET in ['SCOPA', 'CORIELL', 'PRECEPT']:
    for SEX in ['ALL', 'MALE', 'FEMALE']:
        for MODEL in ['ADJ', 'NOADJ']:
            d = combine_lin_outputs(DATASET, SEX, MODEL)
            d.to_csv(f'/data/CARD/projects/dysk_prog/RES/{DATASET}.{SEX}.{MODEL}.Disease_duration.glm.linear.txt', sep='\t', index=False)

In [8]:
def combine_lgs_outputs(DATASET, SEX, MODEL):
    d = pd.DataFrame()
    for i in range(1,23):       
        d1 = pd.read_csv(f'/data/CARD/projects/dysk_prog/logistic/{DATASET}.{SEX}.{MODEL}.chr{i}.Dyskinesia.glm.logistic', sep='\t', low_memory=False)
        d2 = pd.read_csv(f'/data/CARD/projects/dysk_prog/logistic/{DATASET}.{SEX}.{MODEL}.chr{i}.afreq', sep='\t', low_memory=False)
        if d1.shape[0] != d2.shape[0]:
            print(f'ERROR for {DATASET}.{SEX}.{MODEL}.chr{i}')
        else:
            print(f'{DATASET}.{SEX}.{MODEL}.chr{i}')
            d1['ALT_Frq'] = d2.ALT_FREQS
            OR = pd.to_numeric(d1.OR, errors='coerce')
            d1['OR'] = OR.where(d1.ALT==d1.A1, other=1/OR)
            d1['Z_STAT'] = d1.Z_STAT.where(d1.ALT==d1.A1, other=-d1.Z_STAT)
            L95 = pd.to_numeric(d1.L95, errors='coerce')
            U95 = pd.to_numeric(d1.U95, errors='coerce')
            LL = L95.where(d1.ALT==d1.A1, other=1/U95)
            UL = U95.where(d1.ALT==d1.A1, other=1/L95)
            d1['L95'] = LL
            d1['U95'] = UL
            d1 = d1.drop(columns=['A1'])
            d = d.append(d1)
    return(d)

In [ ]:
for DATASET in ['SCOPA']:
    for SEX in ['ALL', 'MALE', 'FEMALE']:
        for MODEL in ['ADJ', 'NOADJ']:
            d = combine_lgs_outputs(DATASET, SEX, MODEL)
            d.to_csv(f'/data/CARD/projects/dysk_prog/RES/{DATASET}.{SEX}.{MODEL}.Dyskinesia.glm.logitstic.txt', sep='\t', index=False)

# Quality check

In [164]:
FILES = glob.glob(f'/data/CARD/projects/dysk_prog/RES/*ALL*')
FILES

['/data/CARD/projects/dysk_prog/RES/PRECEPT.ALL.ADJ.Dyskinesia.cox.txt',
 '/data/CARD/projects/dysk_prog/RES/PRECEPT.ALL.NOADJ.Disease_duration.glm.linear.txt',
 '/data/CARD/projects/dysk_prog/RES/CORIELL.ALL.ADJ.Disease_duration.glm.linear.txt',
 '/data/CARD/projects/dysk_prog/RES/SCOPA.ALL.NOADJ.Dyskinesia.cox.txt',
 '/data/CARD/projects/dysk_prog/RES/PRECEPT.ALL.ADJ.Disease_duration.glm.linear.txt',
 '/data/CARD/projects/dysk_prog/RES/SCOPA.ALL.ADJ.Disease_duration.glm.linear.txt',
 '/data/CARD/projects/dysk_prog/RES/SCOPA.ALL.ADJ.Dyskinesia.cox.txt',
 '/data/CARD/projects/dysk_prog/RES/CORIELL.ALL.ADJ.Dyskinesia.cox.txt',
 '/data/CARD/projects/dysk_prog/RES/CORIELL.ALL.NOADJ.Dyskinesia.cox.txt',
 '/data/CARD/projects/dysk_prog/RES/SCOPA.ALL.NOADJ.Disease_duration.glm.linear.txt',
 '/data/CARD/projects/dysk_prog/RES/CORIELL.ALL.NOADJ.Disease_duration.glm.linear.txt',
 '/data/CARD/projects/dysk_prog/RES/SCOPA.ALL.NOADJ.Dyskinesia.glm.logitstic.txt',
 '/data/CARD/projects/dysk_prog/RE

In [165]:
with open('swarm_qc.txt', 'w') as f:
    for FILE in FILES:
        f.write(f'Rscript --vanilla QQMH.txt {FILE} lambda.txt\n' )

In [167]:
submitMultiJobs(['swarm_qc.txt'], modules='R/4.1.0', time='0:30:00', mem=12, go=True)

['30283944\n']

In [166]:
t = checkJobStatus(['30283944'])

=========== index=0: jobID=27626023: swarm_qc.txt  ===============
N of Jobs: 7
N_of not_COMPLETED: 0
First 5 of longest Runtime job
        Jobid Partition      State Nodes CPUs  Walltime   Runtime  \
5  27626023_4     quick  COMPLETED     1    2  01:00:00  00:09:19   
6  27626023_5     quick  COMPLETED     1    2  01:00:00  00:09:09   
7  27626023_6     quick  COMPLETED     1    2  01:00:00  00:08:56   
3  27626023_2     quick  COMPLETED     1    2  01:00:00  00:08:38   
2  27626023_1     quick  COMPLETED     1    2  01:00:00  00:08:24   

        MemReq MemUsed Nodelist  
5  24.0GB/node   9.4GB   cn2589  
6  24.0GB/node  10.4GB   cn2589  
7  24.0GB/node   9.4GB   cn2589  
3  24.0GB/node  10.4GB   cn2686  
2  24.0GB/node   9.8GB   cn2686  
Pending jobs[]


# Compress folders

after compress folders, do the following to download the files

        scp helix.nih.gov:/data/CARD/projects/dysk_prog/target.tar.gz  /Users/iwakih2/Downloads
        scp helix.nih.gov:/data/CARD/projects/dysk_prog/md5sum_for*  /Users/iwakih2/Downloads
        scp helix.nih.gov:/data/CARD/projects/dysk_prog/RES.tar.gz  /Users/iwakih2/Downloads

In [153]:
%%bash
cd /data/CARD/projects/dysk_prog/target2
tar -zcf target.tar.gz result/
mv target.tar.gz ../target.tar.gz

In [154]:
%%bash
cd /data/CARD/projects/dysk_prog
tar -zcf RES.tar.gz RES/

In [155]:
!ls /data/CARD/projects/dysk_prog/

RES	    RES_old  logistic		 surv		target2
RES.tar.gz  linear   md5sum_for_RES.txt  target.tar.gz


In [157]:
!md5sum /data/CARD/projects/dysk_prog/RES.tar.gz > /data/CARD/projects/dysk_prog/md5sum_for_RES.txt
!md5sum /data/CARD/projects/dysk_prog/target.tar.gz > /data/CARD/projects/dysk_prog/md5sum_for_target.txt

In [158]:
!ls /data/CARD/projects/dysk_prog/

RES	    RES_old  logistic		 md5sum_for_target.txt	target.tar.gz
RES.tar.gz  linear   md5sum_for_RES.txt  surv			target2


In [160]:
!scp  /data/CARD/projects/dysk_prog/target.tar.gz NIALNG-02178851:/Users/iwakih2/Downloads

ssh: Could not resolve hostname nialng-02178851: Name or service not known
lost connection


/gpfs/gsfs9/users/iwakih2/dyskinesia


# Additional Analysis 1

Ever vs Never Dyskinesia analysis of logistic regressions

In [ ]:
for DATASET in ['SCOPA', 'CORIELL', 'PRECEPT']:
    for SEX in ['MALE', 'FEMALE', 'ALL']:
        

In [23]:
# logistic
!mkdir -p /data/CARD/projects/dysk_prog/logistic_en
a = []

for DATASET in ['SCOPA', 'CORIELL', 'PRECEPT']:
    for i in range(1,23):
        CHRNUM = f'chr{i}'
        script1 = [f"""\
plink2 \
 --vcf '/data/CARD/PD/imputed_data/{DATASET}/{CHRNUM}.dose.vcf.gz' 'dosage=DS' \
 --var-filter PASS . GENOTYPED \
 --extract-if-info R2'>'=0.3 \
 --mac 2 \
 --make-pgen --out /data/CARD/projects/dysk_prog/logistic_en/{DATASET}.{CHRNUM} \
 --threads 2 \
"""]
    
        for SEX in ['MALE', 'FEMALE', 'ALL']:
            for MODEL in ['ADJ', 'NOADJ']:

                # define COV

                if MODEL=='NOADJ':
                    COVS = 'PC1,PC2,PC3'
                elif SEX=='ALL':
                    COVS = 'PD_AAO,Sex,HY,Disease_duration,PC1,PC2,PC3'
                else:
                    COVS = 'PD_AAO,HY,Disease_duration,PC1,PC2,PC3'

                script1.append(f"""\
plink2 --pfile /data/CARD/projects/dysk_prog/logistic_en/{DATASET}.{CHRNUM} \
 --glm 'no-firth' 'hide-covar' \
 --freq \
 --covar-variance-standardize \
 --ci 0.95 \
 --pheno data/{DATASET}_{SEX}_en.txt 'iid-only' --pheno-name Dyskinesia \
 --covar data/{DATASET}_{SEX}_en.txt 'iid-only' --covar-name {COVS} \
 --out /data/CARD/projects/dysk_prog/logistic_en/{DATASET}.{SEX}.{MODEL}.{CHRNUM} \
 --threads 2 \
""")

        a.append(' ; '.join(script1))

with open('swarm_logistics_en.txt', 'w') as f:
    f.write('\n'.join(a))

In [24]:
submitMultiJobs(['swarm_logistics_en.txt'], modules='plink/2.3-alpha', time='0:20:00', mem=8, go=True) # bundle next time

['34309884\n']

In [26]:
t = checkJobStatus(['34309884'])

=========== index=0: jobID=34309884: swarm_logistics_en.txt  ===============
N of Jobs: 33
N_of not_COMPLETED: 0
First 5 of longest Runtime job
          Jobid Partition      State Nodes CPUs  Walltime   Runtime  \
1    34309884_0     quick  COMPLETED     1    2  00:20:00  00:05:54   
12  34309884_11     quick  COMPLETED     1    2  00:20:00  00:05:26   
2    34309884_1     quick  COMPLETED     1    2  00:20:00  00:05:18   
23  34309884_22     quick  COMPLETED     1    2  00:20:00  00:04:59   
3    34309884_2     quick  COMPLETED     1    2  00:20:00  00:04:39   

         MemReq MemUsed Nodelist  
1   16.0GB/node   2.1GB   cn2714  
12  16.0GB/node   2.6GB   cn2874  
2   16.0GB/node   2.1GB   cn2714  
23  16.0GB/node   2.3GB   cn2876  
3   16.0GB/node   2.1GB   cn2714  
Pending jobs[]


# Additional Analysis 2
Ever vs Never (Target analysis)

In [ ]:
# Survival Aanalysis - Target
!mkdir -p /data/CARD/projects/dysk_prog/target2/result_en
for DATASET in ['SCOPA', 'CORIELL', 'PRECEPT']:
    infofolder=f"/data/LNG/iwakih2/dataset/{DATASET}/maf01rsq3_20Kcut"
    FILES = [f"/data/CARD/projects/dysk_prog/target2/{DATASET}.all.txt"]

    for FILE in FILES:
        for SEX in ['ALL', 'MALE', 'FEMALE']:
            for MODEL in ['ADJ', 'NOADJ']:

                # define COV

                if MODEL=='NOADJ':
                    COVS = 'PC1+PC2+PC3'
                    
                elif SEX=='ALL':
                    COVS = 'PD_AAO+Sex+HY+PC1+PC2+PC3'
                else:
                    COVS = 'PD_AAO+HY+PC1+PC2+PC3'
                
                # CORIELL doesn't have HY
                if DATASET=='CORIELL':
                    COVS = COVS.replace('HY+','')
                                
                # logistic
                COVS=COVS + '+Disease_duration'
                line = f"Rscript --vanilla logis_target.R {MODEL};Dyskinesia;{COVS};{FILE};data/{DATASET}_{SEX}_en.txt;/data/CARD/projects/dysk_prog/target2/result_en"
                print(line)
                t=submitTerminal(line)

In [6]:
%%bash
cd /data/CARD/projects/dysk_prog/target2
tar -zcf target_en.tar.gz result_en/
mv target_en.tar.gz ../target_en.tar.gz

In [9]:
def combine_lgs_outputs2(DATASET, SEX, MODEL):
    d = pd.DataFrame()
    for i in range(1,23):       
        d1 = pd.read_csv(f'/data/CARD/projects/dysk_prog/logistic_en/{DATASET}.{SEX}.{MODEL}.chr{i}.Dyskinesia.glm.logistic', sep='\t', low_memory=False)
        d2 = pd.read_csv(f'/data/CARD/projects/dysk_prog/logistic_en/{DATASET}.{SEX}.{MODEL}.chr{i}.afreq', sep='\t', low_memory=False)
        if d1.shape[0] != d2.shape[0]:
            print(f'ERROR for {DATASET}.{SEX}.{MODEL}.chr{i}')
        else:
            print(f'{DATASET}.{SEX}.{MODEL}.chr{i}')
            d1['ALT_Frq'] = d2.ALT_FREQS
            OR = pd.to_numeric(d1.OR, errors='coerce')
            d1['OR'] = OR.where(d1.ALT==d1.A1, other=1/OR)
            d1['Z_STAT'] = d1.Z_STAT.where(d1.ALT==d1.A1, other=-d1.Z_STAT)
            L95 = pd.to_numeric(d1.L95, errors='coerce')
            U95 = pd.to_numeric(d1.U95, errors='coerce')
            LL = L95.where(d1.ALT==d1.A1, other=1/U95)
            UL = U95.where(d1.ALT==d1.A1, other=1/L95)
            d1['L95'] = LL
            d1['U95'] = UL
            d1 = d1.drop(columns=['A1'])
            d = d.append(d1)
    return(d)

In [ ]:
!mkdir -p /data/CARD/projects/dysk_prog/RES_en
for DATASET in ['SCOPA', 'CORIELL', 'PRECEPT']:
    for SEX in ['ALL', 'MALE', 'FEMALE']:
        for MODEL in ['ADJ', 'NOADJ']:
            d = combine_lgs_outputs2(DATASET, SEX, MODEL)
            d.to_csv(f'/data/CARD/projects/dysk_prog/RES_en/{DATASET}.{SEX}.{MODEL}.Dyskinesia.glm.logitstic.txt', sep='\t', index=False)

In [33]:
%%bash
cd /data/CARD/projects/dysk_prog
tar -zcf RES_en.tar.gz RES_en/

In [ ]:
# !ls /data/CARD/projects/dysk_prog/logistic_en/SCOPA.ALL.ADJ.chr*